## Import libraries

In [1]:
from google.colab import drive
from pathlib import Path
from matplotlib import pyplot as plt 
import pandas as pd
import numpy as np
import time
import os
import csv
import concurrent.futures

## Utility functions

### Create annot and load descriptors

In [2]:
def create_annot(path):
  image_list = list(Path(path).glob('*/*.jpg'))
  # the identity name is in the path (the name of the parent directory)
  names_list = [i.parent.name for i in image_list]  # get the identity of each image
  # keep info in a pandas DataFrame
  annot = pd.DataFrame({'identity': names_list, 'image_path': image_list})
  return annot

def concatenate_annots(list_of_paths):
  concat_annot = pd.DataFrame()
  with concurrent.futures.ThreadPoolExecutor() as executor:
    annots = [executor.submit(create_annot, path) for path in list_of_paths]
    for annot in annots:
      new_annot = annot.result()
      concat_annot = concat_annot.append(new_annot, ignore_index = True)
    return concat_annot

In [3]:
def load_descriptors(path):
  with open(path, 'rb') as file:
    return np.load(file)

def concatenate_descriptors(list_of_paths):
  concat_descriptors = None
  with concurrent.futures.ThreadPoolExecutor() as executor:
    descriptors = [executor.submit(load_descriptors, path) for path in list_of_paths]
    for descriptor in descriptors:
      new_descriptor = descriptor.result()
      if concat_descriptors is None:
        concat_descriptors = new_descriptor
      else:
        concat_descriptors = np.concatenate([concat_descriptors, new_descriptor])
    return concat_descriptors

### Create pivots

In [4]:
def generate_pivots(descriptors, n, strategy="rnd"):
  if strategy == "kMED":
    kmedoids = sklearn_extra.cluster.KMedoids(n_clusters=n).fit(descriptors)
    return kmedoids.cluster_centers_
  if strategy != "rnd":
    print(strategy, "was not implemented. Random pivots were returned")
  pivots_id = np.random.choice(np.arange(len(descriptors)), size=n)
  return descriptors[pivots_id]

def generate_list_of_pivots(descriptors, t, n, strategy="rnd"):
  list_of_pivots = []
  with concurrent.futures.ThreadPoolExecutor() as executor:
    pivots = [executor.submit(generate_pivots, descriptors, n, strategy) for i in range(t)]
    for pivot in concurrent.futures.as_completed(pivots):
      new_pivot = pivot.result()
      list_of_pivots.append(new_pivot)
    return list_of_pivots

### Save test results

In [5]:
def save_results(dir, file_name, results):
  with open(os.path.join(dir, file_name +".csv"), 'w') as f:
    writer = csv.writer(f)
    # write the header
    writer.writerow(["CLASS", "AP", "QUERY TIME"])
    # write the data
    for r in results:
      writer.writerow(r) 

## Test Performance

In [6]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


### Create annot and load descriptors for the database

In [8]:
db_annot = concatenate_annots(['/content/drive/MyDrive/CV_Birds/train', '/content/drive/MyDrive/CV_Birds/mirflickr25k'])
db_annot

,identity,image_path
0,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
1,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
2,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
3,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
4,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
...,...,...
72327,mirflickr,/content/drive/MyDrive/CV_Birds/mirflickr25k/m...
72328,mirflickr,/content/drive/MyDrive/CV_Birds/mirflickr25k/m...
72329,mirflickr,/content/drive/MyDrive/CV_Birds/mirflickr25k/m...
72330,mirflickr,/content/drive/MyDrive/CV_Birds/mirflickr25k/m...


In [9]:
db_descriptors = concatenate_descriptors(['/content/drive/MyDrive/CV_Birds/features/training/AutoEncoder/512to128withPace64_feature_extraction.npy','/content/drive/MyDrive/CV_Birds/features/distractor/AutoEncoder/512to128withPace64_feature_extraction.npy'])
db_descriptors.shape

(72332, 128)

### Create annot and load descriptors for the test set

In [10]:
query_annot = create_annot('/content/drive/MyDrive/CV_Birds/test')
query_annot

,identity,image_path
0,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/test/AFRICAN C...
1,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/test/AFRICAN C...
2,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/test/AFRICAN C...
3,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/test/AFRICAN C...
4,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/test/AFRICAN C...
...,...,...
1620,YELLOW HEADED BLACKBIRD,/content/drive/MyDrive/CV_Birds/test/YELLOW HE...
1621,YELLOW HEADED BLACKBIRD,/content/drive/MyDrive/CV_Birds/test/YELLOW HE...
1622,YELLOW HEADED BLACKBIRD,/content/drive/MyDrive/CV_Birds/test/YELLOW HE...
1623,YELLOW HEADED BLACKBIRD,/content/drive/MyDrive/CV_Birds/test/YELLOW HE...


In [11]:
query_descriptors = load_descriptors('/content/drive/MyDrive/CV_Birds/features/test/AutoEncoder/512to128withPace64_feature_extraction.npy')
query_descriptors.shape

(1625, 128)

To run our tests we select only the first image of each species within the test set. Please note that within the test set we have 5 images per species.

In [12]:
queries_indexes = [x for x in range(325*5) if x%5 == 0]

### Create PP-Index

In [13]:
def get_descriptor_from_id(id_object):
  return db_descriptors[id_object]

In [14]:
%cd "/content/drive/MyDrive/CV_Birds/Notebooks/PP-Index"
%run PPIndex.ipynb

/content/drive/.shortcut-targets-by-id/1rI5YNBuaSlCB__w522WEkHjw-nFuvIo0/CV_Birds/Notebooks/PP-Index


In [15]:
# generate pivots
list_of_pivots = generate_list_of_pivots(db_descriptors, t=3, n=40, strategy="rnd")

In [16]:
# cosine forest
cosine_forest = PrefixForest(list_of_pivots, length=3, distance_metric='cosine', base_directory="/content/cosine", forest_file='forest_structure')

[WARNING]: Tree is still empty
[WARNING]: Tree is still empty
[WARNING]: Tree is still empty
[WARNING]: Forest is still empty


In [17]:
cosine_forest.insert_objects_into_forest(range(len(db_descriptors)))
cosine_forest.save()

Inserting objects intoInserting objects into tree2 ...
 tree1 ...
Inserting objects into tree3 ...
--- Insertion into tree tree1 completed: 246.163 seconds ---
--- Insertion into tree tree2 completed: 247.429 seconds ---
--- Insertion into tree tree3 completed: 247.944 seconds ---


In [20]:
!cp /content/cosine/tree1* /content/drive/MyDrive/CV_Birds/indexes/feature_extraction/forest/cosine/
!cp /content/cosine/tree2* /content/drive/MyDrive/CV_Birds/indexes/feature_extraction/forest/cosine/
!cp /content/cosine/tree3* /content/drive/MyDrive/CV_Birds/indexes/feature_extraction/forest/cosine/
!cp /content/cosine/forest* /content/drive/MyDrive/CV_Birds/indexes/feature_extraction/forest/cosine/

In [18]:
# euclidean forest
euclidean_forest = PrefixForest(list_of_pivots, length=3, distance_metric='euclidean', base_directory="/content/euclidean", forest_file='forest_structure')

[WARNING]: Tree is still empty
[WARNING]: Tree is still empty
[WARNING]: Tree is still empty
[WARNING]: Forest is still empty


In [19]:
euclidean_forest.insert_objects_into_forest(range(len(db_descriptors)))
euclidean_forest.save()

Inserting objects into tree1 ...
Inserting objects into tree2 ...
Inserting objects into tree3 ...
--- Insertion into tree tree2 completed: 203.524 seconds ---
--- Insertion into tree tree1 completed: 204.587 seconds ---
--- Insertion into tree tree3 completed: 204.922 seconds ---


In [21]:
!cp /content/euclidean/tree1* /content/drive/MyDrive/CV_Birds/indexes/feature_extraction/forest/euclidean/
!cp /content/euclidean/tree2* /content/drive/MyDrive/CV_Birds/indexes/feature_extraction/forest/euclidean/
!cp /content/euclidean/tree3* /content/drive/MyDrive/CV_Birds/indexes/feature_extraction/forest/euclidean/
!cp /content/euclidean/forest* /content/drive/MyDrive/CV_Birds/indexes/feature_extraction/forest/euclidean/

### Compute mAP

In [ ]:
birds_db = db_annot.loc[db_annot['identity'] != 'mirflickr']
counts = birds_db.groupby('identity').count()
print("Minimum number of images per species:", int(counts.min()))
print("Maximum number of images per species:", int(counts.max()))
print("Average number of images:", float(counts.sum()/325))

Minimum number of images per species: 116
Maximum number of images per species: 249
Average number of images: 145.63692307692307


Since at most we have 249 images per species, we use $n=250$.

In [22]:
n = 250

The formula for Average Precision is the following:

> $AP@n=\frac{1}{GTP}\sum_{k=1}^{n}P@k×rel@k$

where $GTP$ refers to the total number of ground truth positives, $n$ refers to the total number of images we are interested in, $P@k$ refers to the precision@k and $rel@k$ is a relevance function. 

The relevance function is an indicator function which equals 1 if the document at rank $k$ is relevant and equals to 0 otherwise.

In [23]:
def compute_ap(query_index, retrieved_ids):
  query_identity = query_annot['identity'][query_index]
  print(query_index//5, query_identity)
  GTP = len(db_annot.loc[db_annot['identity'] == query_identity])
  relevant = 0
  precision_summation = 0
  for k, id in enumerate(retrieved_ids):
    if db_annot['identity'][id] == query_identity: # relevant result
      relevant = relevant + 1
      precision_at_k = relevant/(k+1)
      precision_summation = precision_summation + precision_at_k
  return (query_identity, precision_summation/GTP)

For each query, $Q$, we can calculate a corresponding $AP$. Then, the $mAP$ is simply the mean of all the queries that were made.
> $mAP = \frac{1}{N}\sum_{i=1}^{N}AP_i$

In our case, $N=325$ (one query per species)

#### Simple forest

##### Cosine

In [24]:
def cosine_forest_queries(query_index, n):
  start_time = time.time()
  ids, distances = cosine_forest.find_nearest_neighbors(query_descriptors[query_index], n)
  end_time = time.time()
  ids = ids.tolist()
  return compute_ap(query_index, ids) + (end_time - start_time,)

In [25]:
aps = []
for query_index in queries_indexes:
  aps.append(cosine_forest_queries(query_index, n))

0 AFRICAN CROWNED CRANE
1 AFRICAN FIREFINCH
2 ALBATROSS
3 ALEXANDRINE PARAKEET
4 AMERICAN AVOCET
5 AMERICAN BITTERN
6 AMERICAN COOT
7 AMERICAN GOLDFINCH
8 AMERICAN KESTREL
9 AMERICAN PIPIT
10 AMERICAN REDSTART
11 ANHINGA
12 ANNAS HUMMINGBIRD
13 ANTBIRD
14 ARARIPE MANAKIN
15 ASIAN CRESTED IBIS
16 BALD EAGLE
17 BALD IBIS
18 BALI STARLING
19 BALTIMORE ORIOLE
20 BANANAQUIT
21 BANDED BROADBILL
22 BANDED PITA
23 BAR-TAILED GODWIT
24 BARN OWL
25 BARN SWALLOW
26 BARRED PUFFBIRD
27 BAY-BREASTED WARBLER
28 BEARDED BARBET
29 BEARDED BELLBIRD
30 BEARDED REEDLING
31 BELTED KINGFISHER
32 BIRD OF PARADISE
33 BLACK & YELLOW bROADBILL
34 BLACK BAZA
35 BLACK FRANCOLIN
36 BLACK SKIMMER
37 BLACK SWAN
38 BLACK TAIL CRAKE
39 BLACK THROATED BUSHTIT
40 BLACK THROATED WARBLER
41 BLACK VULTURE
42 BLACK-CAPPED CHICKADEE
43 BLACK-NECKED GREBE
44 BLACK-THROATED SPARROW
45 BLACKBURNIAM WARBLER
46 BLONDE CRESTED WOODPECKER
47 BLUE COAU
48 BLUE GROUSE
49 BLUE HERON
50 BLUE THROATED TOUCANET
51 BOBOLINK
52 BORNEAN BRI

In [26]:
aps

[('AFRICAN CROWNED CRANE', 0.4871392365857213, 1.7077667713165283),
 ('AFRICAN FIREFINCH', 0.48921337183453134, 1.5674233436584473),
 ('ALBATROSS', 0.3323020581621314, 1.6189348697662354),
 ('ALEXANDRINE PARAKEET', 0.12760908275614158, 1.6247572898864746),
 ('AMERICAN AVOCET', 0.9798759376707415, 1.6918013095855713),
 ('AMERICAN BITTERN', 0.9674146259129393, 1.858832836151123),
 ('AMERICAN COOT', 0.9623135246624153, 1.483835220336914),
 ('AMERICAN GOLDFINCH', 0.6333613743620747, 1.6985664367675781),
 ('AMERICAN KESTREL', 0.1564935902354035, 1.532801628112793),
 ('AMERICAN PIPIT', 0.5231209682394761, 1.463942527770996),
 ('AMERICAN REDSTART', 0.1274486908680866, 1.6877272129058838),
 ('ANHINGA', 0.22166294541382783, 1.5218799114227295),
 ('ANNAS HUMMINGBIRD', 0.30605205227965143, 1.546281337738037),
 ('ANTBIRD', 0.07555656991193695, 1.6394681930541992),
 ('ARARIPE MANAKIN', 0.8938207536159749, 1.5848267078399658),
 ('ASIAN CRESTED IBIS', 0.468528471400323, 1.6087000370025635),
 ('BALD E

In [27]:
ap_at_n = np.array([ap[1] for ap in aps])
query_time = np.array(([ap[2] for ap in aps]))

In [28]:
mAP_at_n = np.mean(ap_at_n, axis=0)
avg_query_time = np.mean(query_time, axis=0)
print("mAP:", mAP_at_n)
print("avg. query time: ", avg_query_time)

mAP: 0.45993759936336964
avg. query time:  1.632660296513484


In [29]:
save_results('/content/drive/MyDrive/CV_Birds/performance/fine_tuning/index/AutoEncoder', 'AE_FE_forest_cosine_results', aps)

##### Euclidean

In [30]:
def euclidean_forest_queries(query_index, n):
  start_time = time.time()
  ids, distances = euclidean_forest.find_nearest_neighbors(query_descriptors[query_index], n)
  end_time = time.time()
  ids = ids.tolist()
  return compute_ap(query_index, ids) + (end_time - start_time,)

In [31]:
aps = []
for query_index in queries_indexes:
  aps.append(euclidean_forest_queries(query_index, n))

0 AFRICAN CROWNED CRANE
1 AFRICAN FIREFINCH
2 ALBATROSS
3 ALEXANDRINE PARAKEET
4 AMERICAN AVOCET
5 AMERICAN BITTERN
6 AMERICAN COOT
7 AMERICAN GOLDFINCH
8 AMERICAN KESTREL
9 AMERICAN PIPIT
10 AMERICAN REDSTART
11 ANHINGA
12 ANNAS HUMMINGBIRD
13 ANTBIRD
14 ARARIPE MANAKIN
15 ASIAN CRESTED IBIS
16 BALD EAGLE
17 BALD IBIS
18 BALI STARLING
19 BALTIMORE ORIOLE
20 BANANAQUIT
21 BANDED BROADBILL
22 BANDED PITA
23 BAR-TAILED GODWIT
24 BARN OWL
25 BARN SWALLOW
26 BARRED PUFFBIRD
27 BAY-BREASTED WARBLER
28 BEARDED BARBET
29 BEARDED BELLBIRD
30 BEARDED REEDLING
31 BELTED KINGFISHER
32 BIRD OF PARADISE
33 BLACK & YELLOW bROADBILL
34 BLACK BAZA
35 BLACK FRANCOLIN
36 BLACK SKIMMER
37 BLACK SWAN
38 BLACK TAIL CRAKE
39 BLACK THROATED BUSHTIT
40 BLACK THROATED WARBLER
41 BLACK VULTURE
42 BLACK-CAPPED CHICKADEE
43 BLACK-NECKED GREBE
44 BLACK-THROATED SPARROW
45 BLACKBURNIAM WARBLER
46 BLONDE CRESTED WOODPECKER
47 BLUE COAU
48 BLUE GROUSE
49 BLUE HERON
50 BLUE THROATED TOUCANET
51 BOBOLINK
52 BORNEAN BRI

In [32]:
aps

[('AFRICAN CROWNED CRANE', 0.21141126911144334, 1.6126554012298584),
 ('AFRICAN FIREFINCH', 0.18615900198078575, 1.791060209274292),
 ('ALBATROSS', 0.3397830559723541, 1.7108304500579834),
 ('ALEXANDRINE PARAKEET', 0.16695101059607523, 1.5154485702514648),
 ('AMERICAN AVOCET', 0.9127821795897645, 1.6777963638305664),
 ('AMERICAN BITTERN', 0.9302997929432848, 1.7244701385498047),
 ('AMERICAN COOT', 0.9221921932365615, 1.7404260635375977),
 ('AMERICAN GOLDFINCH', 0.6945342814057307, 1.7006580829620361),
 ('AMERICAN KESTREL', 0.13836493559729593, 1.686267375946045),
 ('AMERICAN PIPIT', 0.29643175183304676, 1.5743606090545654),
 ('AMERICAN REDSTART', 0.1251829581462053, 1.676053524017334),
 ('ANHINGA', 0.20221575441207962, 1.6317248344421387),
 ('ANNAS HUMMINGBIRD', 0.337200272899353, 1.4677708148956299),
 ('ANTBIRD', 0.030362936810239476, 1.5664451122283936),
 ('ARARIPE MANAKIN', 0.6862596610464763, 1.6304619312286377),
 ('ASIAN CRESTED IBIS', 0.4302799943128004, 1.6045212745666504),
 ('B

In [33]:
ap_at_n = np.array([ap[1] for ap in aps])
query_time = np.array(([ap[2] for ap in aps]))

In [34]:
mAP_at_n = np.mean(ap_at_n, axis=0)
avg_query_time = np.mean(query_time, axis=0)
print("mAP:", mAP_at_n)
print("avg. query time: ", avg_query_time)

mAP: 0.35886827445229397
avg. query time:  1.577838483957144


In [35]:
save_results('/content/drive/MyDrive/CV_Birds/performance/fine_tuning/index/AutoEncoder', 'AE_FE_forest_euclidean_results', aps)

#### Forest with query perturbation

##### Cosine

In [36]:
def cosine_pert_forest_queries(query_index, n):
  start_time = time.time()
  ids, distances = cosine_forest.find_nearest_neighbors(query_descriptors[query_index], n, perturbations=3)
  end_time = time.time()
  ids = ids.tolist()
  return compute_ap(query_index, ids) + (end_time - start_time,)

In [37]:
aps = []
for query_index in queries_indexes:
  aps.append(cosine_pert_forest_queries(query_index, n))

0 AFRICAN CROWNED CRANE
1 AFRICAN FIREFINCH
2 ALBATROSS
3 ALEXANDRINE PARAKEET
4 AMERICAN AVOCET
5 AMERICAN BITTERN
6 AMERICAN COOT
7 AMERICAN GOLDFINCH
8 AMERICAN KESTREL
9 AMERICAN PIPIT
10 AMERICAN REDSTART
11 ANHINGA
12 ANNAS HUMMINGBIRD
13 ANTBIRD
14 ARARIPE MANAKIN
15 ASIAN CRESTED IBIS
16 BALD EAGLE
17 BALD IBIS
18 BALI STARLING
19 BALTIMORE ORIOLE
20 BANANAQUIT
21 BANDED BROADBILL
22 BANDED PITA
23 BAR-TAILED GODWIT
24 BARN OWL
25 BARN SWALLOW
26 BARRED PUFFBIRD
27 BAY-BREASTED WARBLER
28 BEARDED BARBET
29 BEARDED BELLBIRD
30 BEARDED REEDLING
31 BELTED KINGFISHER
32 BIRD OF PARADISE
33 BLACK & YELLOW bROADBILL
34 BLACK BAZA
35 BLACK FRANCOLIN
36 BLACK SKIMMER
37 BLACK SWAN
38 BLACK TAIL CRAKE
39 BLACK THROATED BUSHTIT
40 BLACK THROATED WARBLER
41 BLACK VULTURE
42 BLACK-CAPPED CHICKADEE
43 BLACK-NECKED GREBE
44 BLACK-THROATED SPARROW
45 BLACKBURNIAM WARBLER
46 BLONDE CRESTED WOODPECKER
47 BLUE COAU
48 BLUE GROUSE
49 BLUE HERON
50 BLUE THROATED TOUCANET
51 BOBOLINK
52 BORNEAN BRI

In [38]:
aps

[('AFRICAN CROWNED CRANE', 0.7763991716757569, 3.4434821605682373),
 ('AFRICAN FIREFINCH', 0.5277926939450639, 3.38940167427063),
 ('ALBATROSS', 0.3629770006731481, 3.291674852371216),
 ('ALEXANDRINE PARAKEET', 0.5374401962123064, 3.255300521850586),
 ('AMERICAN AVOCET', 0.9796936411767893, 3.3432345390319824),
 ('AMERICAN BITTERN', 0.9672447782115304, 3.492825746536255),
 ('AMERICAN COOT', 0.9582142011978999, 3.2186005115509033),
 ('AMERICAN GOLDFINCH', 0.7361742738162181, 3.6168830394744873),
 ('AMERICAN KESTREL', 0.17268938730033437, 3.2964699268341064),
 ('AMERICAN PIPIT', 0.6247959112090187, 3.0502865314483643),
 ('AMERICAN REDSTART', 0.2111391990838752, 3.8004798889160156),
 ('ANHINGA', 0.241984324737656, 3.152202844619751),
 ('ANNAS HUMMINGBIRD', 0.4229116264672255, 3.1086525917053223),
 ('ANTBIRD', 0.08191132871707556, 3.395031213760376),
 ('ARARIPE MANAKIN', 0.9323349379449309, 3.2468440532684326),
 ('ASIAN CRESTED IBIS', 0.6515051831171373, 3.2295618057250977),
 ('BALD EAGLE'

In [39]:
ap_at_n = np.array([ap[1] for ap in aps])
query_time = np.array(([ap[2] for ap in aps]))

In [40]:
mAP_at_n = np.mean(ap_at_n, axis=0)
avg_query_time = np.mean(query_time, axis=0)
print("mAP:", mAP_at_n)
print("avg. query time: ", avg_query_time)

mAP: 0.5184839518897683
avg. query time:  3.421567313854511


In [41]:
save_results('/content/drive/MyDrive/CV_Birds/performance/fine_tuning/index/AutoEncoder', 'AE_FE_pert_forest_cosine_results', aps)

##### Euclidean

In [42]:
def euclidean_pert_forest_queries(query_index, n):
  start_time = time.time()
  ids, distances = euclidean_forest.find_nearest_neighbors(query_descriptors[query_index], n, perturbations=3)
  end_time = time.time()
  ids = ids.tolist()
  return compute_ap(query_index, ids) + (end_time - start_time,)

In [43]:
aps = []
for query_index in queries_indexes:
  aps.append(euclidean_pert_forest_queries(query_index, n))

0 AFRICAN CROWNED CRANE
1 AFRICAN FIREFINCH
2 ALBATROSS
3 ALEXANDRINE PARAKEET
4 AMERICAN AVOCET
5 AMERICAN BITTERN
6 AMERICAN COOT
7 AMERICAN GOLDFINCH
8 AMERICAN KESTREL
9 AMERICAN PIPIT
10 AMERICAN REDSTART
11 ANHINGA
12 ANNAS HUMMINGBIRD
13 ANTBIRD
14 ARARIPE MANAKIN
15 ASIAN CRESTED IBIS
16 BALD EAGLE
17 BALD IBIS
18 BALI STARLING
19 BALTIMORE ORIOLE
20 BANANAQUIT
21 BANDED BROADBILL
22 BANDED PITA
23 BAR-TAILED GODWIT
24 BARN OWL
25 BARN SWALLOW
26 BARRED PUFFBIRD
27 BAY-BREASTED WARBLER
28 BEARDED BARBET
29 BEARDED BELLBIRD
30 BEARDED REEDLING
31 BELTED KINGFISHER
32 BIRD OF PARADISE
33 BLACK & YELLOW bROADBILL
34 BLACK BAZA
35 BLACK FRANCOLIN
36 BLACK SKIMMER
37 BLACK SWAN
38 BLACK TAIL CRAKE
39 BLACK THROATED BUSHTIT
40 BLACK THROATED WARBLER
41 BLACK VULTURE
42 BLACK-CAPPED CHICKADEE
43 BLACK-NECKED GREBE
44 BLACK-THROATED SPARROW
45 BLACKBURNIAM WARBLER
46 BLONDE CRESTED WOODPECKER
47 BLUE COAU
48 BLUE GROUSE
49 BLUE HERON
50 BLUE THROATED TOUCANET
51 BOBOLINK
52 BORNEAN BRI

In [44]:
aps

[('AFRICAN CROWNED CRANE', 0.2011858834673234, 3.292813301086426),
 ('AFRICAN FIREFINCH', 0.2276312020246452, 3.402472972869873),
 ('ALBATROSS', 0.37098964472546986, 3.3319289684295654),
 ('ALEXANDRINE PARAKEET', 0.3177475296497387, 3.3175649642944336),
 ('AMERICAN AVOCET', 0.9123948579643556, 3.3354947566986084),
 ('AMERICAN BITTERN', 0.9307718915720045, 3.4165353775024414),
 ('AMERICAN COOT', 0.9214820893621146, 3.5231993198394775),
 ('AMERICAN GOLDFINCH', 0.6945342814057307, 3.2312707901000977),
 ('AMERICAN KESTREL', 0.14906768851954405, 3.4631643295288086),
 ('AMERICAN PIPIT', 0.3133789418130626, 3.203216552734375),
 ('AMERICAN REDSTART', 0.17241467612725703, 3.2896358966827393),
 ('ANHINGA', 0.21022883215853114, 3.167919874191284),
 ('ANNAS HUMMINGBIRD', 0.4476835846964979, 3.357484817504883),
 ('ANTBIRD', 0.06970577244498606, 3.3054938316345215),
 ('ARARIPE MANAKIN', 0.7120990078018518, 3.388132333755493),
 ('ASIAN CRESTED IBIS', 0.4694138390286818, 3.15818190574646),
 ('BALD EAG

In [45]:
ap_at_n = np.array([ap[1] for ap in aps])
query_time = np.array(([ap[2] for ap in aps]))

In [46]:
mAP_at_n = np.mean(ap_at_n, axis=0)
avg_query_time = np.mean(query_time, axis=0)
print("mAP:", mAP_at_n)
print("avg. query time: ", avg_query_time)

mAP: 0.40934488120569285
avg. query time:  3.297881835790781


In [47]:
save_results('/content/drive/MyDrive/CV_Birds/performance/fine_tuning/index/AutoEncoder', 'AE_FE_pert_forest_euclidean_results', aps)